# Make Granule List

In [4]:
# import utility function for making granule list
from icelakes.nsidc import make_granule_list
import numpy as np
import pandas as pd
import os

In [18]:
geojson = 'simplified_ANT_1000_East_B-C.geojson'
start_date = '2019-01-02'
end_date = start_date
icesheet = 'AIS'
meltseason = '2018-19'
outname = 'granule_lists/' + icesheet + '_' + meltseason + '_fricker2021.csv'
df = make_granule_list(geojson, start_date, end_date, icesheet, meltseason, outname)
df

Found 1 ATL03 version 006 granules over simplified_ANT_1000_East_B-C.geojson between 2019-01-02 and 2019-01-02.


,granule,geojson,description,geojson_clip,size_mb
0,ATL03_20190102184312_00810210_006_02.h5,geojsons/simplified_ANT_1000_East_B-C.geojson,AIS_2018-19_simplified_ANT_1000_East_B-C,geojsons/ANT_1000_East_B-C.geojson,1447.870142


In [13]:
dates_melling = [
    '2020-07-06',
    '2020-07-17',
    '2020-07-06',
    '2020-07-06',
    '2019-07-16'
]
dfs = []
for i in np.arange(1,6):
    geojson = 'melling2024_lake%d' % i
    start_date = dates_melling[i-1]
    end_date = start_date
    icesheet = 'GrIS'
    meltseason = start_date[:4]
    outname = 'granule_lists/' + icesheet + '_' + meltseason + '_' + geojson.replace('.geojson','') + '.csv'
    df = make_granule_list(geojson, start_date, end_date, icesheet, meltseason, outname)
    dfs.append(df)
df_melling = pd.concat(dfs)
df_melling

Found 1 ATL03 version 006 granules over melling2024_lake1 between 2020-07-06 and 2020-07-06.
Found 1 ATL03 version 006 granules over melling2024_lake2 between 2020-07-17 and 2020-07-17.
Found 1 ATL03 version 006 granules over melling2024_lake3 between 2020-07-06 and 2020-07-06.
Found 1 ATL03 version 006 granules over melling2024_lake4 between 2020-07-06 and 2020-07-06.
Found 1 ATL03 version 006 granules over melling2024_lake5 between 2019-07-16 and 2019-07-16.


,granule,geojson,description,geojson_clip,size_mb
0,ATL03_20200706005932_01630805_006_01.h5,geojsons/melling2024_lake1,GrIS_2020_melling2024_lake1,geojsons/melling2024_lake1,1143.116431
0,ATL03_20200717114945_03380803_006_01.h5,geojsons/melling2024_lake2,GrIS_2020_melling2024_lake2,geojsons/melling2024_lake2,5462.348953
0,ATL03_20200706005932_01630805_006_01.h5,geojsons/melling2024_lake3,GrIS_2020_melling2024_lake3,geojsons/melling2024_lake3,1143.116431
0,ATL03_20200706005932_01630805_006_01.h5,geojsons/melling2024_lake4,GrIS_2020_melling2024_lake4,geojsons/melling2024_lake4,1143.116431
0,ATL03_20190716051841_02770403_006_02.h5,geojsons/melling2024_lake5,GrIS_2019_melling2024_lake5,geojsons/melling2024_lake5,1552.577720


In [2]:
dflist = []
filename_out = 'granule_lists/icelakes-methods.csv'

# Greenland CW
startyears = [2019, 2020]
startday = '05-01'
endday = '09-30'
icesheet = 'GrIS'
geojson = 'simplified_GRE_2000_CW.geojson'

for yr in startyears:
    start_date = '%s-%s' % (yr, startday)
    end_date = '%s-%s' % (yr, endday)
    meltseason = start_date[:4] if start_date[:4]==end_date[:4] else start_date[:4] + '-' + end_date[2:4]
    outname = 'granule_lists/' + icesheet + '_' + meltseason + '_' + geojson.replace('.geojson','') + '.csv'
    df = make_granule_list(geojson, start_date, end_date, icesheet, meltseason, outname)
    dflist.append(df)

# Antarctica B-C
startyears = [2018, 2020]
startday = '11-01'
endday = '03-31'
icesheet = 'AIS'
geojson = 'simplified_ANT_1000_East_B-C.geojson'

for yr in startyears:
    start_date = '%s-%s' % (yr, startday)
    end_date = '%s-%s' % (yr+1, endday)
    meltseason = start_date[:4] if start_date[:4]==end_date[:4] else start_date[:4] + '-' + end_date[2:4]
    outname = 'granule_lists/' + icesheet + '_' + meltseason + '_' + geojson.replace('.geojson','') + '.csv'
    df = make_granule_list(geojson, start_date, end_date, icesheet, meltseason, outname)
    dflist.append(df)


df_all = pd.concat(dflist)
df_all.loc[:, 'description'] = df_all.apply(lambda x: x.loc['description'].replace('simplified_','').replace('GRE_','').replace('ANT_',''), axis=1)
df_all.reset_index(inplace=True, drop=True)
df_all.to_csv(filename_out.replace('.csv', '_size.csv'), header=False, index=False)

print('Number of ganules over Greenland:', np.sum(df_all.apply(lambda x: 'GrIS' in x.loc['description'], axis=1)))
print('Number of ganules over Antarctica:', np.sum(df_all.apply(lambda x: 'AIS' in x.loc['description'], axis=1)))
print('Total number of granules:', len(df_all))
maxrow = df_all.loc[np.argmax(df_all.size_mb),:]
print('Largest granule: %.1f GB, %s, %s' % (maxrow.size_mb/1000, maxrow.granule, maxrow.geojson))
print('Total size: %.2f TB' % (np.sum(df_all.size_mb)/1e6))

df_all_nosize = df_all.drop(columns='size_mb').copy()
df_all_nosize.to_csv(filename_out, header=False, index=False)
df_all_nosize

Found 84 ATL03 version 006 granules over simplified_GRE_2000_CW.geojson between 2019-05-01 and 2019-09-30.
Found 94 ATL03 version 006 granules over simplified_GRE_2000_CW.geojson between 2020-05-01 and 2020-09-30.
Found 132 ATL03 version 006 granules over simplified_ANT_1000_East_B-C.geojson between 2018-11-01 and 2019-03-31.
Found 137 ATL03 version 006 granules over simplified_ANT_1000_East_B-C.geojson between 2020-11-01 and 2021-03-31.
Number of ganules over Greenland: 178
Number of ganules over Antarctica: 269
Total number of granules: 447
Largest granule: 11.2 GB, ATL03_20200524142922_09020703_006_01.h5, geojsons/simplified_GRE_2000_CW.geojson
Total size: 1.15 TB


,granule,geojson,description,geojson_clip
0,ATL03_20190502215043_05290305_006_02.h5,geojsons/simplified_GRE_2000_CW.geojson,GrIS_2019_2000_CW,geojsons/GRE_2000_CW.geojson
1,ATL03_20190503084011_05360303_006_02.h5,geojsons/simplified_GRE_2000_CW.geojson,GrIS_2019_2000_CW,geojsons/GRE_2000_CW.geojson
2,ATL03_20190503212504_05440305_006_02.h5,geojsons/simplified_GRE_2000_CW.geojson,GrIS_2019_2000_CW,geojsons/GRE_2000_CW.geojson
3,ATL03_20190507083151_05970303_006_02.h5,geojsons/simplified_GRE_2000_CW.geojson,GrIS_2019_2000_CW,geojsons/GRE_2000_CW.geojson
4,ATL03_20190507211645_06050305_006_02.h5,geojsons/simplified_GRE_2000_CW.geojson,GrIS_2019_2000_CW,geojsons/GRE_2000_CW.geojson
...,...,...,...,...
442,ATL03_20210327161149_00431112_006_01.h5,geojsons/simplified_ANT_1000_East_B-C.geojson,AIS_2020-21_1000_East_B-C,geojsons/ANT_1000_East_B-C.geojson
443,ATL03_20210330034125_00811110_006_01.h5,geojsons/simplified_ANT_1000_East_B-C.geojson,AIS_2020-21_1000_East_B-C,geojsons/ANT_1000_East_B-C.geojson
444,ATL03_20210330162908_00891112_006_01.h5,geojsons/simplified_ANT_1000_East_B-C.geojson,AIS_2020-21_1000_East_B-C,geojsons/ANT_1000_East_B-C.geojson
445,ATL03_20210331031546_00961110_006_01.h5,geojsons/simplified_ANT_1000_East_B-C.geojson,AIS_2020-21_1000_East_B-C,geojsons/ANT_1000_East_B-C.geojson
